# STAT3009 Project 1
- LAW Yiu Leung Eric 1155149315

## Load the developed methods

In [1]:
import numpy as np
from developed_methods import *

## Pre-process the raw data
- check the `user_id` and `item_id`: mapping `item_id` to a continuous sequence based on `sklean.preprocessing`
- use `sklearn.model_selection.train_test_split` to generate `train` and `test` dataset
- create `train_pair`,`train_rating`, `test_pair`

In [2]:
import pandas as pd
df = pd.read_csv("data/train.csv")
dtest_submit = pd.read_csv("data/test.csv")

## mapping 
from sklearn import preprocessing
le_user = preprocessing.LabelEncoder()
le_user.fit(np.append(df['user_id'], dtest_submit["user_id"]))
df['user_id'] = le_user.transform(df["user_id"])
dtest_submit["user_id"] = le_user.transform(dtest_submit["user_id"])

le_item = preprocessing.LabelEncoder()
le_item.fit(np.append(df['item_id'], dtest_submit["item_id"]))
df["item_id"] = le_item.transform(df["item_id"])
dtest_submit["item_id"] = le_item.transform(dtest_submit["item_id"])

## generate train / test dataset
from sklearn.model_selection import train_test_split
dtrain, dtest = train_test_split(df, test_size=0.33, random_state=42)

## save real ratings for test set for evaluation.
test_rating = np.array(dtest['rating'])

## remove the ratings in the test set to simulate prediction
dtest = dtest.drop(columns='rating')

Create train_pair, train_rating and test_pair

In [3]:
# train_pair, train_rating
train_pair = dtrain[['user_id', 'item_id']].values
train_rating = dtrain['rating'].values

# test_pair
test_pair = dtest[['user_id', 'item_id']].values
n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

# Impelement deep learning with Keras

In [4]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

## Create the model
We embed both users and movies in to 50-dimensional vectors.

The model computes a match score between user and movie embeddings via a dot product, and adds a per-movie and per-user bias.

class LFactorNet(keras.Model):
    def __init__(self, num_users, num_items, embedding_size, **kwargs):
        super(LFactorNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.glb_bias = tf.Variable(0., trainable=True) 
        self.movie_embedding = layers.Embedding(
            num_items,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.movie_bias = layers.Embedding(num_items, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias + self.glb_bias
        return x

In [8]:
model = LFactorNet(num_users=n_user, num_items=n_item, embedding_size=50)


metrics = [
    keras.metrics.MeanAbsoluteError(name='mae'),
    keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=metrics
)

callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_rmse', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

history = model.fit(
    x=train_pair,
    y=train_rating,
    batch_size=64,
    epochs=200,
    verbose=1,
    validation_split=.2,
)

Epoch 1/200
129/129 [==============================] - 2s 3ms/step - loss: 46.5726 - mae: 5.9808 - rmse: 6.7221 - val_loss: 43.9892 - val_mae: 5.7692 - val_rmse: 6.5327
Epoch 2/200
129/129 [==============================] - 0s 2ms/step - loss: 34.2254 - mae: 4.8618 - rmse: 5.7271 - val_loss: 41.1163 - val_mae: 5.4971 - val_rmse: 6.2892
Epoch 3/200
129/129 [==============================] - 0s 2ms/step - loss: 23.2300 - mae: 3.6397 - rmse: 4.6328 - val_loss: 36.5682 - val_mae: 5.0364 - val_rmse: 5.8803
Epoch 4/200
129/129 [==============================] - 0s 2ms/step - loss: 14.9365 - mae: 2.5658 - rmse: 3.5700 - val_loss: 31.1090 - val_mae: 4.4544 - val_rmse: 5.3600
Epoch 5/200
129/129 [==============================] - 0s 2ms/step - loss: 12.0649 - mae: 2.2642 - rmse: 3.1007 - val_loss: 28.3663 - val_mae: 4.1547 - val_rmse: 5.0867
Epoch 6/200
129/129 [==============================] - 0s 2ms/step - loss: 11.5768 - mae: 2.2439 - rmse: 3.0178 - val_loss: 27.5207 - val_mae: 4.0769 - val

In [9]:
## make prediction
pred_rating = model.predict(test_pair).flatten()
print(pred_rating)
print('rmse: LFactorNet: %.3f' %np.sqrt(np.mean((pred_rating - test_rating)**2)))

[6.0982437 5.8481407 4.09268   ... 3.3440409 6.140058  3.8438263]
rmse: LFactorNet: 1.401


In [9]:
pred_rating = model.predict(dtest_submit[["user_id", "item_id"]]).flatten()
print(pred_rating)

[2.6130528 4.703871  5.0403795 ... 3.0259562 8.935659  4.4722023]


In [ ]:
submit = pd.DataFrame({"Id": range(len(dtest_submit)), "rating": pred_rating})
submit.to_csv("predict/submission1.csv", index = False)